## GNN for drug-drug combination

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure  # for the shap plots
import random
import os
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, matthews_corrcoef, precision_score, recall_score, accuracy_score
from sklearn.model_selection import StratifiedGroupKFold, ParameterGrid
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from torch_geometric.data import Data
from torch_geometric.nn.conv import GCNConv, GATConv, TransformerConv
from torch_geometric.nn import MessagePassing
from torch_geometric import seed_everything
from shap import KernelExplainer

In [ ]:
visualizations = "../visualizations"
if not os.path.exists(visualizations):
    os.makedirs(visualizations)

In [ ]:
seed_everything(42)  # random seed for train/test split

### Datasets with new ID

In [ ]:
# the node and edge features specified in Feature_selection.ipynb
nodes = pd.read_csv("../data/gnn_node_features.csv")   # took the op feature out
edges = pd.read_csv("../data/gnn_edge_features.csv")   # took the op feature out

In [ ]:
# convert node and edge features into two lists
def nodes_edges(nodes, edges):
    global edge_features, node_features
    edge_arr = edges.columns[5:len(edges.columns)-1].to_list()  # excluding ID
    edge_features = []
    for i in range(len(edge_arr)):
        edge_features.append(edges[edge_arr[i]])

    node_arr = nodes.columns[2:].to_list()  # excluding ID
    node_features = []
    for i in range(len(node_arr)):
        node_features.append(nodes[node_arr[i]])

nodes_edges(nodes, edges)

In [ ]:
# based on the number of nodes and edges, convert them into tensors 
def dataset():
    x = torch.tensor(node_features, dtype=torch.float32).t()
    edge_index = torch.tensor([edges["drugA_ID"], edges["drugB_ID"]], dtype=torch.int64)
    edge_attr = torch.tensor(edge_features, dtype=torch.float).t()
    edge_labels = torch.tensor(edges[edges.columns[-1]], dtype=torch.long)
    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, edge_labels=edge_labels)
graph_data = dataset()

### From PyTorch Geometric's documentation:
- https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.data.Data.html
- x = Node feature matrix with shape [num_nodes, num_node_features]
- edge_index = Graph connectivity in COO format with shape [2, num_edges]
- edge_attr = Edge feature matrix with shape [num_edges, num_edge_features]
- y = Graph-level or node-level ground-truth labels, but for link classification tasks, use edge_labels
- output: Data(x=[4412, 8], edge_index=[2, 95986], edge_attr=[95986, 6], edge_labels=[95986]) (doubling the edges to ensure the graph is undirected)
- index with 8 node attributes, connectivity, 6 edge attributes and 95986 directed edges

In [ ]:
num_edge_features = graph_data.edge_attr.shape[-1] 
num_edge_features

In [ ]:
graph_data.edge_index.shape

### Graph properties
Check if the graph is directed. If not, then process the data to ensure the undirectedness

In [ ]:
graph_data.has_isolated_nodes()

In [ ]:
graph_data.has_self_loops()

In [ ]:
graph_data.is_directed()  # ensure this outputs False

## Graph architectures

### Set seed for reproducibility

In [ ]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    random.seed(seed)
    torch.use_deterministic_algorithms(True)

set_seed(1234)

### Stratified Group K-Fold cross-validation

### Train/validation/test set

In [ ]:
%%html
<style>
table {float:left}
</style>

### Idea of k-fold cross-validation: example with k=5 with 70% train, 10% validation and 20% test data
The models were initially trained with k=5. However, the GCN didn't classify anything into class 1 during most of the folds, k=10 is used for all models and the 80/10/10 split was used in the end for evaluation. 

First split:
| Fold 1 | Fold 2 |	Fold 3 | Fold 4 | Fold 5 |
| -------- | ------- | ------- |  ------- | ------- | 
| Train/validation | Train/validation | Train/validation | Train/validation | Test |
| Test | Train/validation | Train/validation | Train/validation | Train/validation |
| Train/validation | Test| Train/validation | Train/validation | Train/validation |
| Train/validation | Train/validation | Test | Train/validation |  Train/validation |
| Train/validation | Train/validation | Train/validation | Test | Train/validation |

In the second split, the data looks like this:  
| Fold 1 | Fold 2 |	Fold 3 | Fold 4 | Fold 5 | Fold 6 | Fold 7 | Fold 8
| -------- | ------- | ------- |  ------- | ------- | ------- | ------- | ------- | 
| Train | Train | Train | Train | Train | Train | Train | Validation

The final result: 
| Fold 1 | Fold 2 |	Fold 3 | Fold 4 | Fold 5
| -------- | ------- | ------- |  ------- | ------- | 
| Train | Train | Train | Validation | Test | 
| Test | Train | Train | Train | Validation |
| Validation | Test | Train | Train |  Train |
| Train | Validation | Test | Train | Train |

In [ ]:
# train, validation and test sets 
def kfold_new(train_value, val_value):
    global train, val, test, train_val, train_labels, val_labels, test_labels
    train, val, test, train_val = [], [], [], []
    train_labels, val_labels, test_labels = [], [], []
    
    kf = StratifiedGroupKFold(n_splits=train_value, shuffle=False)
    groups = edges["drugcomb_sorted"].to_list() 
    
    # train/val: 90%, test: 10% -> first split
    for i, (train_val_idx, test_idx) in enumerate(kf.split(graph_data.edge_index.t(), graph_data.edge_labels, groups)): 
        print(f"Fold {i+1}:")
        print(f" Train and Validation: index={train_val_idx[:20]}") 
        
        train_val_groups = np.array(groups)[train_val_idx.astype(int)]
        train_val_y = graph_data.edge_labels[train_val_idx]

        # add the indices and labels
        train_val.append(train_val_idx)
        test.append(test_idx)
        test_labels.append(graph_data.edge_labels[test_idx])
        
        # train: 80%, val: 10% -> second split
        inner_skf = StratifiedGroupKFold(n_splits=val_value, shuffle=False) 
        train_idx, val_idx = next(inner_skf.split(graph_data.edge_index[:, train_val_idx].t(), train_val_y, train_val_groups))    

        # combine train and validation indies
        arr1, arr2 = train_idx, val_idx
        arr = [*arr1, *arr2]
        arr.sort()

        # create dictionary for the mapping
        list1 = arr  # new index
        list2 = train_val_idx  # old index
        d1 = {}
        for i in range(len(list1)):  # everything: train + val
            d1[list1[i]] = list2[i]

        # convert the new to the original indices 
        old_idx = []   
        old_idx_ = []
        for i in range(len(train_idx)):
            old_idx.append(d1.get(train_idx[i])) 

        for i in range(len(val_idx)):
            old_idx_.append(d1.get(val_idx[i])) 

        # check whether the 3 sets have overlapping elements
        """print("Check for any overlap between train and validation")
        print(list(set(old_idx).intersection(old_idx_)))

        print("Check for any overlap between train and test")
        print(list(set(old_idx).intersection(test_idx)))

        print("Check for any overlap between validation and test")
        print(list(set(old_idx_).intersection(test_idx)))"""
 
        print(f"     Train: index={old_idx[:20]}, length={len(old_idx)}") 
        print(f"     Validation: index={old_idx_[:20]}, length={len(old_idx_)}") 
        
        train.append(old_idx)
        train_labels.append(graph_data.edge_labels[old_idx])
        val.append(old_idx_)
        val_labels.append(graph_data.edge_labels[old_idx_])

        print(f" Test:  index={test_idx[:20]}, length={len(test_idx)}")  # 10% of the total
        print("*"*100)

# different combinations
# 80% train, 10% val, 10% test
kfold_new(10,9)

# ------ previous tests -------
# 70% train, 10% val, 20% test 
# kfold_new(5,8)

# 60% train, 20% val, 20% test -> doesn't work that well and almost no labels got classified correctly into class 1
# kfold_new(5,4)

### Graph Convolutional Network 

In [ ]:
class GCN(MessagePassing):  # node features and edge indices only, GCN -> GCN -> Linear -> Softmax probabilities -> Output class
    def __init__(self, dim_in, dim_h1, dim_h2=4):
        super().__init__()
        self.conv1 = GCNConv(dim_in, dim_h1)
        self.conv2 = GCNConv(dim_h1, dim_h2)
        self.linear = Linear(dim_h2, 2) 

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        x = self.propagate(edge_index, x=x)  # message passing
        return x

    def decode(self, z, edge_index):
        out = self.linear(edge_feat)
        out = torch.softmax(out, dim=1)
        return out

    def forward(self, x, edge_index):
        global edge_feat
        z = self.encode(x, edge_index)
        x_src, x_dst = z[edge_index[0]], z[edge_index[1]]
        edge_feat = torch.cat([x_src + x_dst], dim=-1)  # map node embeddings into edge predictions, added them to ensure permutation invariance
        return self.decode(z, edge_index)

### Graph Attention Network

In [ ]:
class GAT(MessagePassing):
    def __init__(self, edge_attr, dim_in, dim_h1, dim_h2=4):
        super().__init__()
        self.gat1 = GATConv(dim_in, dim_h1, edge_dim=edge_attr)
        self.gat2 = GATConv(dim_h1, dim_h2, edge_dim=edge_attr)
        self.linear = Linear(dim_h2 + edge_attr, 2)

    def forward(self, x, edge_index, edge_attr):
        x = self.gat1(x, edge_index, edge_attr=edge_attr).relu()
        x = self.gat2(x, edge_index, edge_attr=edge_attr).relu()

        x = self.propagate(edge_index, x=x, edge_attr=edge_attr)  # add message passing
        x_src, x_dst = x[edge_index[0]], x[edge_index[1]] 
        edge_feat = torch.cat([x_src + x_dst, edge_attr], dim=-1)  # adding edge features into the final prediction
        out = self.linear(edge_feat) 
        out = torch.softmax(out, dim=1)
        return out

### Graph Transformer

In [ ]:
class Transformer(MessagePassing):
    def __init__(self, edge_attr, dim_in, dim_h1, dim_h2=4):
        super().__init__()
        self.gat1 = TransformerConv(dim_in, dim_h1, edge_dim=edge_attr)
        self.gat2 = TransformerConv(dim_h1, dim_h2, edge_dim=edge_attr)
        self.linear = Linear(dim_h2 + edge_attr, 2)

    def forward(self, x, edge_index, edge_attr):
        x = self.gat1(x, edge_index, edge_attr=edge_attr).relu()
        x = self.gat2(x, edge_index, edge_attr=edge_attr).relu()

        x = self.propagate(edge_index, x=x, edge_attr=edge_attr)  # add message passing
        x_src, x_dst = x[edge_index[0]], x[edge_index[1]]
        edge_feat = torch.cat([x_src + x_dst, edge_attr], dim=-1)  # adding edge features into the final prediction
        out = self.linear(edge_feat)
        out = torch.softmax(out, dim=1)
        return out

### Training loop, validation and test data

In [ ]:
def fit(model, lr, weight_decay, weighted_loss, edge_attr, epochs=30):
    global train_prediction, val_pred, train_loss, val_loss, train_acc, val_acc, mcc_final
    train_prediction, val_pred = [], []
    train_loss, val_loss, train_acc, val_acc = [], [], [], []

    weights = torch.tensor([1, weighted_loss])  # class 1 is the minority class so it's higher weighted
    weights = weights.to(torch.float)
    loss_fn = CrossEntropyLoss(weight=weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay) 

    for i in range(len(train)): 
        print(f"********** Fold {i+1} train data: **********")
        train_edge_index = graph_data.edge_index[:, train[i]]
        train_labels = graph_data.edge_labels[train[i]]
        val_edge_index = graph_data.edge_index[:, val[i]]
        val_labels = graph_data.edge_labels[val[i]]
        
        # initialize validation set with another stratified group k fold 
        for epoch in range(epochs + 1):
            model.train()
            optimizer.zero_grad()

            if edge_attr:   # GAT, GT
                train_edge_attr = graph_data.edge_attr[train[i]]
                output = model(graph_data.x, train_edge_index, train_edge_attr)
            else:    # GCN
                output = model(graph_data.x, train_edge_index)
            
            loss = loss_fn(output, train_labels)  # target value is edge classification
            loss.backward()
            optimizer.step()
            
            if epoch % 10 == 0: 
                # evaluate train data
                accuracy = torch.sum(torch.argmax(output, dim=1) == train_labels) / len(train_labels)
                train_prediction_ = output.argmax(1)
                mcc_train = matthews_corrcoef(train_labels, train_prediction_)
                
                print(f"Epoch: {epoch}")
                print(" Train data: ")
                print(f"   Loss: {loss}")
                print(f"   Accuracy: {accuracy}") 
                print(f"   MCC: {mcc_train}")

                # evaluate validation data
                if edge_attr:
                    val_edge_attr = graph_data.edge_attr[val[i]]
                    val_out = model(graph_data.x, val_edge_index, val_edge_attr)
                else:
                    val_out = model(graph_data.x, val_edge_index)
                    
                loss_ = loss_fn(val_out, val_labels)
                val_accuracy = torch.sum(torch.argmax(val_out, dim=1) == val_labels) / len(val_labels)
                val_pred_ = val_out.argmax(1)
                val_mcc = matthews_corrcoef(val_labels, val_pred_)
                
                print("Validation data: ")
                print(f"   Loss: {loss_}")
                print(f"   Accuracy: {val_accuracy}")
                print(f"   MCC: {val_mcc}")

                if epoch == 30:
                    # append train data results
                    train_loss.append(loss.detach().numpy())
                    train_acc.append(accuracy.detach().numpy())
                    train_prediction.append(output.argmax(1))

                    # append validation data results                    
                    val_loss.append(loss_.detach().numpy())
                    val_acc.append(val_accuracy.detach().numpy())
                    val_pred.append(val_out.argmax(1))    

                    cm = confusion_matrix(train_labels, train_prediction[i])
                    cm2 = confusion_matrix(val_labels, val_pred[i])
                    ConfusionMatrixDisplay(cm).plot()  
                    
                    plt.savefig(f"{visualizations}/GCN Fold {i+1} Train.svg")
                    #plt.savefig(f"{visualizations}/GAT Fold {i+1} Train.svg")
                    #plt.savefig(f"{visualizations}/GT Fold {i+1} Train.svg")
                    
                    #print("*"*100)
                    ConfusionMatrixDisplay(cm2).plot()                    
                    plt.savefig(f"{visualizations}/GCN Fold {i+1} Validation.svg")
                    #plt.savefig(f"{visualizations}/GAT Fold {i+1} Validation.svg")
                    #plt.savefig(f"{visualizations}/GT Fold {i+1} Validation.svg")
                    
                # use MCC score to evaluate the model
                if epoch == 30 and i == 9:   # assume that the last epoch and last fold has the best score
                    mcc_final = matthews_corrcoef(val_labels, val_pred[i])  
                    print(f"    MCC Final: {mcc_final}")
        
    return mcc_final

In [ ]:
def test_model(model, edge_attr):
    global pred_test
    pred_test = []

    #weights = torch.tensor([1, weighted_loss])  
    #weights = weights.to(torch.float)
    #loss_fn = CrossEntropyLoss(weight=weights)

    for i in range(len(test)):
        print(f"********** Fold {i+1} test data: **********")
        test_edge_index = graph_data.edge_index[:, test[i]]
        test_labels = graph_data.edge_labels[test[i]]

        if edge_attr:   # GAT and GT
            test_edge_attr = graph_data.edge_attr[test[i]]
            output = model(graph_data.x, test_edge_index, test_edge_attr)
        else:   # GCN
            output = model(graph_data.x, test_edge_index)
        
        pred_test.append(output.argmax(1))
        #loss = loss_fn(output, test_labels)
        accuracy = torch.sum(torch.argmax(output, dim=1) == test_labels) / len(test_labels)
        mcc_test = matthews_corrcoef(test_labels, pred_test[i])
       
        print("Test data:")
        #print(f"   Loss: {loss}")
        print(f"   Accuracy: {accuracy}")
        print(f"   MCC: {mcc_test}")

In [ ]:
# use this for every GNN model
"""
Initial parameter grid
param_grid = {
    'hidden_channels': [8, 16, 32, 64],
    'learning_rate': [0.01, 0.001, 0.0001],
    'weight_decay': [5e-4, 1e-4],
    'weighted_loss': [80, 100, 120, 140, 160, 180, 200, 220],
}
"""

# final parameter grid
param_grid = {
    'hidden_channels': [8, 16],
    'learning_rate': [0.001, 0.0001],
    'weight_decay': [5e-4, 1e-4],
    'weighted_loss': [100, 120, 140, 160, 180, 200],
}

grid = ParameterGrid(param_grid)

In [ ]:
# finding best hyperparameters
best_val_acc = -np.inf
best_params = None

for params in grid:
    print(f"Hyperparameters: weighted_loss={params['weighted_loss']}, hidden_channels={params['hidden_channels']}, learning_rate={params['learning_rate']}, weight_decay={params['weight_decay']}")

    # choose a model from below:
    #model = GCN(graph_data.num_node_features, params['hidden_channels'])
    #model = GAT(num_edge_features, graph_data.num_features, params['hidden_channels'])
    model = Transformer(num_edge_features, graph_data.num_features, params['hidden_channels'])

    # choose a training function from below:
    # GCN
    #fit(model, params['learning_rate'], params['weight_decay'], params['weighted_loss'], False)
    # GAT and GT
    fit(model, params['learning_rate'], params['weight_decay'], params['weighted_loss'], True)
    
    # find best hyperparameters using the MCC score
    if mcc_final > best_val_acc:
        best_val_acc = mcc_final
        best_params = params

print(f"Best Hyperparameters: {best_params}, Best Validation Accuracy: {best_val_acc}")

In [ ]:
# best hyperparameters for GCN
# Best Hyperparameters: {'hidden_channels': 16, 'learning_rate': 0.0001, 'weight_decay': 0.0005, 'weighted_loss': 120}, 
# Best Validation Accuracy: 0.08698340010113292

In [ ]:
# best hyperparameters for GAT
# Best Hyperparameters: {'hidden_channels': 8, 'learning_rate': 0.001, 'weight_decay': 0.0001, 'weighted_loss': 120}, 
# Best Validation Accuracy: 0.03473767142951595

In [ ]:
# best hyperparameters for GT
# Best Hyperparameters: {'hidden_channels': 16, 'learning_rate': 0.001, 'weight_decay': 0.0001, 'weighted_loss': 100}, 
# Best Validation Accuracy: 0.06322532092824903

### Test the models using the best hyperparameters obtained from ParameterGrid

In [ ]:
model = GCN(graph_data.num_node_features, 16)
fit(model, 0.001, 0.0005, 110, False) 

In [ ]:
test_model(model, False)

In [ ]:
gat_model = GAT(num_edge_features, graph_data.num_features, 8)

In [ ]:
fit(gat_model, 0.001, 0.0005, 90, True) 

In [ ]:
test_model(gat_model, True)

In [ ]:
transformer = Transformer(num_edge_features, graph_data.num_features, 8)

In [ ]:
fit(transformer, 0.001, 0.0005, 100, True) 

In [ ]:
test_model(transformer, True)

### Evaluation 

In [ ]:
def cm_test(n):
    for i in range(n):
        cm = confusion_matrix(test_labels[i], pred_test[i])
        mcc = matthews_corrcoef(test_labels[i], pred_test[i])
        print(mcc)
        ConfusionMatrixDisplay(cm).plot()
        
        plt.savefig(f"{visualizations}/Confusion matrix GCN - fold {i+1} Test.png")
        #plt.savefig(f"{visualizations}/Confusion matrix GAT - fold {i+1} Test.png")
        #plt.savefig(f"{visualizations}/Confusion matrix GT - fold {i+1} Test.png")

cm_test(10)

In [ ]:
def evaluation(test_arr, pred_test):
    global data
    mcc_gcn, precision_gcn, recall_gcn, accuracy_gcn = [], [], [], []
    for i in range(10):
        mcc = matthews_corrcoef(test_arr[i], pred_test[i])
        precision = precision_score(test_arr[i], pred_test[i])
        recall = recall_score(test_arr[i], pred_test[i])
        accuracy = accuracy_score(test_arr[i], pred_test[i])

        mcc_gcn.append(mcc)
        precision_gcn.append(precision)
        recall_gcn.append(recall)
        accuracy_gcn.append(accuracy)
    data = [mcc_gcn, precision_gcn, recall_gcn, accuracy_gcn]
    return data

# export the train/validation/test evaluation
#evaluation(train_labels, train_prediction)
#evaluation(val_labels, val_pred)
evaluation(test_labels, pred_test)

In [ ]:
# export the evaluation scores into a csv file
eval_columns = ["MCC", "Precision", "Recall", "Accuracy"]

def export_results(model, data, columns):
    evaluation = "../evaluation"
    if not os.path.exists(evaluation):
        os.makedirs(evaluation)
        
    df = pd.DataFrame()
    for i in range(len(data)):
        df[columns[i]] = pd.Series(data[i])
    
    df.index += 1 
    df.to_csv(f"{evaluation}/{model}.csv", index_label="ID")
    return df

export_results("GCN_Test", data, eval_columns) 
#export_results("GAT_Test", data, eval_columns) 
#export_results("GT_Test", data, eval_columns) 